In [8]:
from qiskit_algorithms import SamplingVQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit.result import QuasiDistribution
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.primitives import Sampler
from qiskit import QuantumCircuit
from qiskit.circuit.library import TwoLocal
import numpy as np


def processing(invocation_input):
    mu = invocation_input['mu']
    sigma = invocation_input['sigma']
    global qp
    qp = to_po(mu, sigma, budget=invocation_input['budget'], q=invocation_input['q'])
    return QuantumCircuit(2)


def post_processing(job_result):
    result = vqe(qp)
    print_result(result, qp)
    return result.x


def vqe(qp):
    num_assets = len(qp.variables)
    cobyla = COBYLA()
    cobyla.set_options(maxiter=1000)
    svqe_mes = SamplingVQE(sampler=Sampler(), ansatz=TwoLocal(num_assets, "ry", "cz", reps=3, entanglement="full"), optimizer=cobyla)
    svqe = MinimumEigenOptimizer(svqe_mes)
    result = svqe.solve(qp)
    return result

def to_po(mu, sigma, **kwargs):
    q = kwargs['q']
    budget = kwargs['budget']
    if budget is None:
        budget = len(sigma) // 2  # num_assets // 2
    portfolio = PortfolioOptimization(
        expected_returns=mu, covariances=sigma, risk_factor=q, budget=budget
    )
    qp = portfolio.to_quadratic_program()
    return qp
  
def print_result(result, qp):
    selection = result.x
    value = result.fval
    print("Optimal: selection {}, value {:.4f}".format(selection, value))

    eigenstate = result.min_eigen_solver_result.eigenstate
    probabilities = (
        eigenstate.binary_probabilities()
        if isinstance(eigenstate, QuasiDistribution)
        else {k: np.abs(v) ** 2 for k, v in eigenstate.to_dict().items()}
    )
    print("\n----------------- Full result ---------------------")
    print("selection\tvalue\t\tprobability")
    print("---------------------------------------------------")
    probabilities = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)

    for k, v in probabilities:
        x = np.array([int(i) for i in list(reversed(k))])
        value = qp.objective.evaluate(x)
        print("%10s\t%.4f\t\t%.4f" % (x, value, v))

In [ ]:
qiskit-algorithms
qiskit-finance
qiskit-optimization
numpy

In [9]:
import data
mu, sigma, asset_list = data.get_mu_sigma('asset.csv', 4)
invocation_input = {
    "q": 0.5,
    "mu": [
        -0.0035346909221779968,
        -0.004927625199511683,
        -0.00675916981502751,
        0.0040393323478814455
    ],
    "sigma": [
        [
            0.00308759,
            0.0010389,
            0.00067355,
            0.0007161
        ],
        [
            0.0010389,
            0.00183111,
            0.00131204,
            0.00130782
        ],
        [
            0.00067355,
            0.00131204,
            0.00221604,
            0.001021
        ],
        [
            0.0007161,
            0.00130782,
            0.001021,
            0.0019567
        ]
    ],
    "budget": 2
}
job_result = processing(invocation_input)
post_processing(job_result)

Optimal: selection [1. 0. 0. 1.], value 0.0027

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
 [0 0 1 1]	0.0058		0.5260
 [1 0 0 1]	0.0027		0.3733
 [0 1 1 0]	0.0150		0.0800
 [1 1 0 0]	0.0120		0.0201
 [0 1 0 1]	0.0041		0.0005
 [1 1 0 1]	0.0109		0.0000
 [0 1 1 1]	0.0143		0.0000
 [0 1 0 0]	0.0058		0.0000
 [0 0 1 0]	0.0079		0.0000
 [1 1 1 0]	0.0218		0.0000
 [1 0 0 0]	0.0051		0.0000
 [1 0 1 1]	0.0123		0.0000
 [1 0 1 0]	0.0136		0.0000
 [0 0 0 1]	-0.0031		0.0000
 [0 0 0 0]	0.0000		0.0000
 [1 1 1 1]	0.0218		0.0000


array([1., 0., 0., 1.])

In [7]:
qp = to_po(mu, sigma, budget=2, q=0.5)
len(qp.variables)

4